In [13]:
import pandas as pd 
import numpy as np
import plotly.express as px

# --- Importações de Scraping e Componentes ---
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
import time # Mantido apenas como uma pausa de segurança

# --- Configuração Estável do WebDriver (Importante) ---
try:
    from webdriver_manager.chrome import ChromeDriverManager
    def get_chrome_driver(options):
        # Gerencia e instala o driver automaticamente
        return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
except ImportError:
    # Fallback caso o 'webdriver-manager' não esteja instalado
    def get_chrome_driver(options):
        print("Atenção: webdriver-manager não disponível. Certifique-se de que o ChromeDriver está no seu PATH.")
        return webdriver.Chrome(options=options)

# 1. PREPARAÇÃO DOS DADOS
city_data = pd.read_csv('city_data_cleaned.csv')
city_data_coords = city_data.copy()
city_data_coords['Latitude'] = np.nan
city_data_coords['Longitude'] = np.nan


# 2. FUNÇÃO PRINCIPAL DE WEB SCRAPING
def scrape_city_coordinates_robust(df):
    
    # Configuração do WebDriver
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless") # Descomente para rodar sem abrir a janela
    
    try:
        driver = get_chrome_driver(options)
    except Exception as e:
        print(f"ERRO CRÍTICO: Não foi possível iniciar o Chrome. Detalhes: {e}")
        return df_result
    
    base_url = "https://en.wikipedia.org/wiki/Main_Page"
    df_result = df.copy()
    
    print("Início do processo de Scraping. Será usado um tempo de espera forçado para garantir a leitura.")
    
    for index, row in df_result.iterrows():
        city = row['City']
        country = row['Country']
        query = f"{city}, {country}" 
        
        try:
            driver.get(base_url)
            
            # 2.1. LOCALIZAR BARRA DE PESQUISA
            # Uso de seletor CSS e espera para garantir que está clicável
            SEARCH_INPUT_SELECTOR = "#searchInput" 
            search_input = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, SEARCH_INPUT_SELECTOR))
            )
            search_input.clear()
            search_input.send_keys(query)
            search_input.send_keys(Keys.RETURN)
            
            # Pausa de segurança para garantir o redirecionamento após a pesquisa
            time.sleep(1)
            
            # 2.2. ESPERA FORÇADA PELA COORDENADA (O FIX principal)
            # O script irá ESPERAR até 10 segundos até que o elemento com a classe "geo" apareça.
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "geo"))
            )
            
            # 2.3. EXTRAÇÃO COM BEAUTIFULSOUP
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Busca pelo elemento que contém a coordenada decimal
            geo_tag = soup.find("span", {"class": "geo"})
            
            if geo_tag:
                coords_text = geo_tag.get_text()
                if ';' in coords_text:
                    lat, lon = coords_text.split(';')
                    df_result.at[index, 'Latitude'] = float(lat.strip())
                    df_result.at[index, 'Longitude'] = float(lon.strip())
                    print(f"✅ Sucesso: {city}")
                else:
                    print(f"⚠️ Aviso: Coordenadas encontradas, mas em formato inválido para {city}.")
            else:
                print(f"❌ Falha: Elemento de coordenadas não encontrado para {city}.")
                
        except Exception as e:
            print(f"❌ Erro ao processar {city}. Detalhes: Elemento de Coordenada não apareceu a tempo.")

    driver.quit()
    print("\nScraping concluído. A verificar resultados...")
    return df_result

# --- EXECUÇÃO: FASE 2 (Scraping) ---
city_data_with_coords = scrape_city_coordinates_robust(city_data_coords)
city_data_with_coords.to_csv('city_data_with_coords.csv', index=False)
print(f"Total de cidades com coordenadas encontradas: {city_data_with_coords['Latitude'].count()}")


# 3. ADVANCED TOPIC: VISUALIZAÇÃO DO MAPA
map_df = city_data_with_coords.dropna(subset=['Latitude', 'Longitude'])

if not map_df.empty:
    print("\nGerando Mapa Interativo...")
    fig_map = px.scatter_mapbox(
        map_df,
        lat="Latitude",
        lon="Longitude",
        hover_name="City",
        # Configurações do Tooltip (Popup) conforme as diretrizes
        hover_data={
            "Latitude": False, "Longitude": False,
            "Country": True,
            "Population": ':,.0f', 
            "Average Monthly Salary": ':,.0f €', 
            "Average Cost of Living": ':,.0f €'  
        },
        color="Average Monthly Salary", 
        size="Population",              
        color_continuous_scale="Viridis",
        zoom=3, 
        height=700,
        title="Mapa Interativo de Cidades Europeias"
    )

    fig_map.update_layout(mapbox_style="open-street-map")
    fig_map.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    fig_map.show()
else:
    print("Geração do mapa ignorada: Dados de coordenadas insuficientes.")

Atenção: webdriver-manager não disponível. Certifique-se de que o ChromeDriver está no seu PATH.
Início do processo de Scraping. Será usado um tempo de espera forçado para garantir a leitura.
❌ Erro ao processar Salzburg. Detalhes: Elemento de Coordenada não apareceu a tempo.
❌ Erro ao processar Vienna. Detalhes: Elemento de Coordenada não apareceu a tempo.
✅ Sucesso: Antwerp
❌ Erro ao processar Bruges. Detalhes: Elemento de Coordenada não apareceu a tempo.
❌ Erro ao processar Brussels. Detalhes: Elemento de Coordenada não apareceu a tempo.
❌ Erro ao processar Gent. Detalhes: Elemento de Coordenada não apareceu a tempo.
❌ Erro ao processar Dobrich. Detalhes: Elemento de Coordenada não apareceu a tempo.
❌ Erro ao processar Sofia. Detalhes: Elemento de Coordenada não apareceu a tempo.
❌ Erro ao processar Split. Detalhes: Elemento de Coordenada não apareceu a tempo.
❌ Erro ao processar Zagreb. Detalhes: Elemento de Coordenada não apareceu a tempo.
❌ Erro ao processar Lefkosia. Detalhes: E

C:\Users\afons\AppData\Local\Temp\ipykernel_8352\1405219024.py:115: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

